In [1]:
import numpy as np
import pandas as pd
import collections 
data = pd.read_csv("train.csv")
data.head()

,Date,Temp9am,Temp3pm,MinTemp,MaxTemp,Rainfall,RainToday,Evaporation,Sunshine,WindGustDir,...,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,ID
0,01-02-13,20.7,20.9,19.5,22.4,15.6,Yes,6.2,0.0,W,...,SSW,17,20,92,84,1017.6,1017.4,8,8,1
1,02-02-13,22.4,24.8,19.5,25.6,6.0,Yes,3.4,2.7,W,...,E,9,13,83,73,1017.9,1016.4,7,7,2
2,03-02-13,23.5,23.0,21.6,24.5,6.6,Yes,2.4,0.1,W,...,ESE,17,2,88,86,1016.7,1015.6,7,8,3
3,04-02-13,21.4,20.9,20.2,22.8,18.8,Yes,2.2,0.0,W,...,E,22,20,83,90,1014.2,1011.8,8,8,4
4,05-02-13,22.5,25.5,19.7,25.7,77.4,Yes,4.8,0.0,W,...,W,11,6,88,74,1008.3,1004.8,8,8,5


In [2]:
print(data.shape)
for item in data.columns:
    print(item)

(2616, 22)
Date
Temp9am
Temp3pm
MinTemp
MaxTemp
Rainfall
RainToday
Evaporation
Sunshine
WindGustDir
WindGustSpeed
WindDir9am
WindDir3pm
WindSpeed9am
WindSpeed3pm
Humidity9am
Humidity3pm
Pressure9am
Pressure3pm
Cloud9am
Cloud3pm
ID


In [3]:
null_columns = []
for item in data.columns:

    if data[item].isnull().any():
        null_columns.append(item)

print(null_columns)
#No null columns :(

[]


In [4]:
data.columns = [
    'date',
    'temp_9am',
    'temp_3pm',
    'min_temp',
    'max_temp',
    'rain_fall',
    'rain_today',
    'evaporation',
    'sunshine',
    'wind_gust_dir',
    'wind_gust_speed',
    'wind_dir_9am',
    'wind_dir_3pm',
    'wind_speed_9am',
    'wind_speed_3pm',
    'humidity_9am',
    'humidity3pm',
    'pressure_9am',
    'pressure_3pm',
    'cloud_9am',
    'cloud_3pm',
    'id'
]
data = data.drop('id' , axis = 1)
data.columns

Index(['date', 'temp_9am', 'temp_3pm', 'min_temp', 'max_temp', 'rain_fall',
       'rain_today', 'evaporation', 'sunshine', 'wind_gust_dir',
       'wind_gust_speed', 'wind_dir_9am', 'wind_dir_3pm', 'wind_speed_9am',
       'wind_speed_3pm', 'humidity_9am', 'humidity3pm', 'pressure_9am',
       'pressure_3pm', 'cloud_9am', 'cloud_3pm'],
      dtype='object')

In [5]:
X = data.drop('rain_today' , axis = 1).copy()
Y = data['rain_today'].copy()

In [6]:
from datetime import datetime

def date_to_days(date : str) -> int:

    time = datetime.strptime(date , "%d-%m-%y")
    ref_from = datetime(2000 , 1 , 1)
    days = (time - ref_from).days

    return days

X['date'] = X['date'].map(date_to_days)
X.head()

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_dir,wind_gust_speed,wind_dir_9am,wind_dir_3pm,wind_speed_9am,wind_speed_3pm,humidity_9am,humidity3pm,pressure_9am,pressure_3pm,cloud_9am,cloud_3pm
0,4780,20.7,20.9,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,17,20,92,84,1017.6,1017.4,8,8
1,4781,22.4,24.8,19.5,25.6,6.0,3.4,2.7,W,41,W,E,9,13,83,73,1017.9,1016.4,7,7
2,4782,23.5,23.0,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,17,2,88,86,1016.7,1015.6,7,8
3,4783,21.4,20.9,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,22,20,83,90,1014.2,1011.8,8,8
4,4784,22.5,25.5,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,11,6,88,74,1008.3,1004.8,8,8


In [7]:
#We will going to store catagorical columns for CatBoost
cat_columns = []

for item in X.columns:

    if X[item].dtype == object:
        cat_columns.append(item)

for item in cat_columns: print(item)

wind_gust_dir
wind_dir_9am
wind_dir_3pm


In [8]:
for item in cat_columns:
    print(f"{item} : {X[item].unique()}")
    print(f"len : {len(X[item].unique())}")

wind_gust_dir : ['W' 'NNW' 'WNW' 'ENE' 'NNE' 'NW' 'SSE' 'NE' 'ESE' 'WSW' 'SE' 'SW' 'N' 'E'
 'SSW' 'S']
len : 16
wind_dir_9am : ['S' 'W' 'ESE' 'NNE' 'SSW' 'WNW' 'N' 'SW' 'SE' 'SSE' 'WSW' 'E' 'ENE' 'NW'
 'NNW' 'NE']
len : 16
wind_dir_3pm : ['SSW' 'E' 'ESE' 'W' 'ENE' 'S' 'SE' 'SSE' 'NE' 'NNE' 'NNW' 'NW' 'WNW' 'N'
 'WSW' 'SW']
len : 16


In [9]:
from sklearn.preprocessing import OneHotEncoder
#We also store a copy of data for catboost before it is hot encoded.
X_cat_copy = X.copy()

encoder = OneHotEncoder(sparse_output = False , 
                        categories = 'auto', 
                        handle_unknown= 'error',
                        max_categories = 32)

encoder.fit(X[cat_columns])
cat_ohed = pd.DataFrame(encoder.transform(X[cat_columns]))

In [10]:
X = X.drop(cat_columns , axis = 1)
X_encoded = pd.concat([X , cat_ohed] , axis = 1)
X_encoded.head()

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_speed,wind_speed_9am,...,38,39,40,41,42,43,44,45,46,47
0,4780,20.7,20.9,19.5,22.4,15.6,6.2,0.0,41,17,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,4781,22.4,24.8,19.5,25.6,6.0,3.4,2.7,41,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4782,23.5,23.0,21.6,24.5,6.6,2.4,0.1,41,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4783,21.4,20.9,20.2,22.8,18.8,2.2,0.0,41,22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4784,22.5,25.5,19.7,25.7,77.4,4.8,0.0,41,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
Y_MAPED: pd.Series = Y.map({'Yes' : 1 , 'No' : 0})
Y_MAPED

0       1
1       1
2       1
3       1
4       1
       ..
2611    0
2612    0
2613    0
2614    0
2615    0
Name: rain_today, Length: 2616, dtype: int64

In [12]:
print(Y_MAPED.sum(axis = 0)/len(Y_MAPED))
#So , obviously we need careful stratification before train_test_split()

0.25840978593272174


In [13]:
#Now before feature engineering ,first I want to reduce the number of
# dimensions of X
from sklearn.decomposition import PCA

params_pca: dict = {
    'n_components' : 0.95,
    'whiten'       : False,
    'random_state' : 12,
    'svd_solver'   : 'full',
}

pca_model = PCA(**params_pca)
pca_model.fit(X)
X_reduced_raw_values: np.ndarray = pca_model.transform(X)



In [14]:
col_len: int = X_reduced_raw_values.shape[1]
new_columns: list = [f"PCA_FEATURE_{idx}".lower() for idx in range(col_len)]
print(new_columns)
X_reduced: pd.DataFrame = pd.DataFrame(data = X_reduced_raw_values , columns = new_columns)
X_reduced

['pca_feature_0']


,pca_feature_0
0,-1307.567090
1,-1306.544648
2,-1305.572877
3,-1304.565725
4,-1303.554320
...,...
2611,1303.489310
2612,1304.545363
2613,1305.540950
2614,1306.536836


In [15]:
#Uhhh , it seems we have lost too much data . Even though , we captured
#95% of the total variance, the 1 single dimension is very worse.
#We should ignore that.
from sklearn.model_selection import train_test_split

X_ROOT_TRAIN: pd.DataFrame
X_ROOT_TEST: pd.DataFrame
Y_ROOT_TRAIN: pd.DataFrame
Y_ROOT_TEST: pd.DataFrame

tts_params: dict = {
    'test_size' : float(0.2),
    'train_size' : float(0.8),
    'random_state' : int(13),
    'shuffle' : bool(True),
    'stratify'  : Y_MAPED,
}

X_ROOT_TRAIN , X_ROOT_TEST, Y_ROOT_TRAIN , Y_ROOT_TEST = train_test_split(
    X_encoded,
    Y_MAPED,
    **tts_params
)



In [16]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import optuna

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
class enum_model:
    def __init__(self , model_name: str = "XGBM") -> any:
        if not model_name in ["XGBM" , "CB" , "LGBM"]:
            raise ValueError(
                "Error! Model has to be: [XGBM , CB , LGBM]")
        self.model: str = model_name
    def getModelType(self) -> str:
        return self.model
    

def _obj_for_xgbm(trial: optuna.Trial) -> float:

    xgbm_params: dict = {
    #===============#
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'tree_method': 'hist',
    'scale_pos_weight': (Y_ROOT_TRAIN == 0).sum()/(Y_ROOT_TRAIN == 1).sum(),
    'random_state': 13,
    'n_jobs': -1,
    'verbosity': 1,
    #===============#
    
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
    'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.7, 1.0),
    'colsample_bynode': trial.suggest_float('colsample_bynode', 0.7, 1.0),
    'gamma': trial.suggest_float('gamma', 0.3, 1),
    'learning_rate': trial.suggest_float('learning_rate', 0.01 , 0.05),
    'max_depth': trial.suggest_int('max_depth', 3, 8),
    'min_child_weight': trial.suggest_int('min_child_weight', 0.3 , 0.9),
    'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
    'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 1.5),
    'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 1.5),
    'subsample': trial.suggest_float('subsample', 0.8, 1.0),
    }


    __model: XGBClassifier = XGBClassifier(**xgbm_params)
    validator: StratifiedKFold = StratifiedKFold(n_splits = 10 , 
                                                  shuffle = True , 
                                                  random_state = 13)
    score: float = cross_val_score(__model, 
                                   X_encoded, 
                                   Y_MAPED,
                                   scoring = "accuracy",
                                   n_jobs = -1,
                                   verbose =  0,
                                   cv = validator).mean()
    return score

def _obj_for_lgbm(trial: optuna.Trial) -> float:
     lgbm_params: dict = {
         #==================#
        'random_state': 13,
        'scale_pos_weight' : (Y_ROOT_TRAIN == 0).sum()/(Y_ROOT_TRAIN == 1).sum(),
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'objective' : 'binary',
         #==================#

        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3 , 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01 , 0.05),
        'max_depth': trial.suggest_int('max_depth', 3 , 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 8),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.3, 0.8),
        'n_estimators': trial.suggest_int('n_estimators', 1000 , 2000),
        'max_bin': trial.suggest_int('max_bin', 32 , 64),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1 , 1.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1 , 1.5),
        'subsample': trial.suggest_float('subsample', 0.8 , 1),
     }

     __model: LGBMClassifier = LGBMClassifier(**lgbm_params)
     validator: StratifiedKFold = StratifiedKFold(n_splits = 10 , 
                                                  shuffle = True , 
                                                  random_state = 13)
     score: float = cross_val_score(__model, 
                                    X_encoded, 
                                    Y_MAPED,
                                    scoring = "accuracy",
                                    n_jobs = -1,
                                    verbose =  0,
                                    cv = validator).mean()
     return score
def _obj_for_cb(trial: optuna.Trial) -> float:
    
    cb_params: dict = {
        #=================#
        'verbose': 0,
        'auto_class_weights' : "Balanced",
        'random_state': 13,
        'boosting_type' : 'Plain',
        'task_type': 'CPU',  
        #=================#


        'iterations': trial.suggest_int('iterations', 1000 , 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'depth': trial.suggest_int('depth', 3, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 1.5),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 0.3),
        'random_strength': trial.suggest_float('random_strength', 0.2, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 4),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 0, 3),
        
    }

    __model: CatBoostClassifier = CatBoostClassifier(**cb_params)
    validator: StratifiedKFold = StratifiedKFold(
        n_splits=10, 
        shuffle=True, 
        random_state=13
        )
    score = cross_val_score(
        __model,
        X_encoded,
        Y_MAPED,
        scoring='accuracy',
        cv=validator,
        n_jobs=-1,
        verbose=0
    ).mean()
    
    return score

class func:
    pass
def objective(model_name: enum_model) -> func:
    match(model_name.getModelType()):
        case "XGBM": return _obj_for_xgbm
        case "LGBM": return _obj_for_lgbm
        case "CB"  : return _obj_for_cb

In [18]:
xgbm_obj: func = objective(enum_model("XGBM"))

storage_path_xgbm: str = "sqlite:///xgbm_study.db"
study_xgbm: optuna.Study = optuna.study.create_study(
    study_name = "xgbm_study",
    storage  = storage_path_xgbm,
    direction= 'maximize',
    )
study_xgbm.optimize(
    func = xgbm_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = False,
    
)


[I 2025-12-25 16:06:54,054] A new study created in RDB with name: xgbm_study
[I 2025-12-25 16:06:59,451] Trial 6 finished with value: 0.8329311807200724 and parameters: {'colsample_bytree': 0.700194855958183, 'colsample_bylevel': 0.8366266956813858, 'colsample_bynode': 0.7373573180754142, 'gamma': 0.6917558290608926, 'learning_rate': 0.04672230323354876, 'max_depth': 8, 'min_child_weight': 0, 'n_estimators': 1337, 'reg_alpha': 0.255025034813558, 'reg_lambda': 1.09846781017725, 'subsample': 0.9451319144666979}. Best is trial 6 with value: 0.8329311807200724.
[I 2025-12-25 16:07:02,100] Trial 5 finished with value: 0.826432394489778 and parameters: {'colsample_bytree': 0.7682327993361835, 'colsample_bylevel': 0.913831309059201, 'colsample_bynode': 0.9837329653765468, 'gamma': 0.45201396910651975, 'learning_rate': 0.028563806194069306, 'max_depth': 8, 'min_child_weight': 0, 'n_estimators': 1376, 'reg_alpha': 1.2747299874222902, 'reg_lambda': 0.4953069750963357, 'subsample': 0.819530259264

In [19]:
lgbm_obj: func = objective(enum_model("LGBM"))

storage_path_lgbm: str = "sqlite:///lgbm_study.db"
study_lgbm = optuna.create_study(
    study_name = "lgbm_study",
    storage  = storage_path_lgbm,
    direction= 'maximize',
)
study_lgbm.optimize(
    func = lgbm_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = False
)

[I 2025-12-25 16:08:10,462] A new study created in RDB with name: lgbm_study
[I 2025-12-25 16:08:19,086] Trial 4 finished with value: 0.8199394577520399 and parameters: {'colsample_bytree': 0.3431038552676605, 'learning_rate': 0.041001702092872405, 'max_depth': 7, 'min_child_samples': 3, 'min_child_weight': 0.601487006456602, 'n_estimators': 1652, 'max_bin': 49, 'reg_alpha': 1.4466506183344159, 'reg_lambda': 1.1796907232023568, 'subsample': 0.8557853869328909}. Best is trial 4 with value: 0.8199394577520399.
[I 2025-12-25 16:08:22,643] Trial 1 finished with value: 0.8260551022198825 and parameters: {'colsample_bytree': 0.6757372688837648, 'learning_rate': 0.04202498938988612, 'max_depth': 6, 'min_child_samples': 3, 'min_child_weight': 0.5740428417970086, 'n_estimators': 1976, 'max_bin': 45, 'reg_alpha': 1.0573220460459873, 'reg_lambda': 0.8451738091060396, 'subsample': 0.8083425413144347}. Best is trial 1 with value: 0.8260551022198825.
[I 2025-12-25 16:08:25,570] Trial 0 finished with

In [20]:
cb_obj: func = objective(enum_model("CB"))
storage_path_cb: str = "sqlite:///cb_study.db"
study_cb = optuna.create_study(
    study_name = "cb_study",
    storage  = storage_path_cb,
    direction= 'maximize',
    load_if_exists = True
)
study_cb.optimize(
    func = cb_obj,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = False 
)

[I 2025-12-25 16:12:19,226] A new study created in RDB with name: cb_study
[I 2025-12-25 16:12:31,773] Trial 4 finished with value: 0.8302448012634904 and parameters: {'iterations': 1055, 'learning_rate': 0.043010373356293694, 'depth': 5, 'l2_leaf_reg': 1.3754822347590832, 'bagging_temperature': 0.03255420478150632, 'random_strength': 0.5582574944524468, 'min_data_in_leaf': 2, 'subsample': 0.7976576865964419, 'max_ctr_complexity': 3}. Best is trial 4 with value: 0.8302448012634904.
[I 2025-12-25 16:13:18,390] Trial 5 finished with value: 0.8279620367933083 and parameters: {'iterations': 1933, 'learning_rate': 0.04764776348346921, 'depth': 8, 'l2_leaf_reg': 0.7250641995105248, 'bagging_temperature': 0.047922409886452445, 'random_strength': 0.337638564769554, 'min_data_in_leaf': 2, 'subsample': 0.9517782806834247, 'max_ctr_complexity': 2}. Best is trial 4 with value: 0.8302448012634904.
[I 2025-12-25 16:13:31,835] Trial 3 finished with value: 0.824130619168787 and parameters: {'iteration

In [22]:
best_params_xgbm: dict = study_xgbm.best_params.copy()
BASE_MODEL_XGBM: XGBClassifier = XGBClassifier(
    **best_params_xgbm , 
    random_state = 13,
    objective = 'binary:logistic',
    booster= 'gbtree',
    tree_method = 'hist',
    n_jobs = -1,
    scale_pos_weight = (Y_ROOT_TRAIN == 0).sum()/(Y_ROOT_TRAIN == 1).sum()
    )
BASE_MODEL_XGBM.fit(X_ROOT_TRAIN , 
                    Y_ROOT_TRAIN , 
                    eval_set = [(X_ROOT_TEST , Y_ROOT_TEST)],
                    verbose = True,
                    )

[0]	validation_0-logloss:0.67670
[1]	validation_0-logloss:0.66337
[2]	validation_0-logloss:0.64832
[3]	validation_0-logloss:0.63644
[4]	validation_0-logloss:0.62499
[5]	validation_0-logloss:0.61247
[6]	validation_0-logloss:0.60318
[7]	validation_0-logloss:0.59493
[8]	validation_0-logloss:0.58570
[9]	validation_0-logloss:0.57534
[10]	validation_0-logloss:0.56652
[11]	validation_0-logloss:0.55957
[12]	validation_0-logloss:0.55284
[13]	validation_0-logloss:0.54472
[14]	validation_0-logloss:0.53711
[15]	validation_0-logloss:0.53105
[16]	validation_0-logloss:0.52523
[17]	validation_0-logloss:0.52067
[18]	validation_0-logloss:0.51475
[19]	validation_0-logloss:0.50971
[20]	validation_0-logloss:0.50519
[21]	validation_0-logloss:0.50114
[22]	validation_0-logloss:0.49633
[23]	validation_0-logloss:0.49123
[24]	validation_0-logloss:0.48692
[25]	validation_0-logloss:0.48341
[26]	validation_0-logloss:0.47986
[27]	validation_0-logloss:0.47670
[28]	validation_0-logloss:0.47302
[29]	validation_0-loglos

,objective,'binary:logistic'
,base_score,None
,booster,'gbtree'
,callbacks,None
,colsample_bylevel,0.7608426444691376
,colsample_bynode,0.7964685684175228
,colsample_bytree,0.6629506354849434
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [23]:
best_params_lgbm: dict = study_lgbm.best_params.copy()
BASE_MODEL_LGBM: LGBMClassifier = LGBMClassifier(
    **best_params_lgbm ,
    n_jobs = -1,
    random_state = 13,
    boosting_type = 'gbdt',
    objective = 'binary',
    scale_pos_weight = (Y_ROOT_TRAIN == 0).sum()/(Y_ROOT_TRAIN == 1).sum()
    )
BASE_MODEL_LGBM.fit(
    X_ROOT_TRAIN,
    Y_ROOT_TRAIN,
    eval_set = [(X_ROOT_TEST, Y_ROOT_TEST)],
)

[LightGBM] [Info] Number of positive: 541, number of negative: 1551
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000357 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 752
[LightGBM] [Info] Number of data points in the train set: 2092, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258604 -> initscore=-1.053236
[LightGBM] [Info] Start training from score -1.053236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,7
,learning_rate,0.04347414519375591
,n_estimators,1729
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.3023021334034122
,min_child_samples,5


In [24]:
best_params_cb: dict = study_cb.best_params.copy()
BASE_CB_MODEL: CatBoostClassifier = CatBoostClassifier(
    **best_params_cb,
    verbose = 1,
    auto_class_weights = "Balanced",
    random_state = 13,
    task_type = "CPU",
    boosting_type = 'Plain'
    )
BASE_CB_MODEL.fit(
    X_ROOT_TRAIN,
    Y_ROOT_TRAIN,
    eval_set = [(X_ROOT_TEST , Y_ROOT_TEST)],
    verbose = 1
)

0:	learn: 0.6856310	test: 0.6879249	best: 0.6879249 (0)	total: 150ms	remaining: 2m 48s
1:	learn: 0.6790761	test: 0.6831120	best: 0.6831120 (1)	total: 154ms	remaining: 1m 26s
2:	learn: 0.6712247	test: 0.6776106	best: 0.6776106 (2)	total: 157ms	remaining: 59s
3:	learn: 0.6636492	test: 0.6724871	best: 0.6724871 (3)	total: 161ms	remaining: 45.1s
4:	learn: 0.6568483	test: 0.6673685	best: 0.6673685 (4)	total: 164ms	remaining: 36.8s
5:	learn: 0.6494051	test: 0.6615250	best: 0.6615250 (5)	total: 168ms	remaining: 31.4s
6:	learn: 0.6426725	test: 0.6560672	best: 0.6560672 (6)	total: 172ms	remaining: 27.5s
7:	learn: 0.6362942	test: 0.6514951	best: 0.6514951 (7)	total: 175ms	remaining: 24.5s
8:	learn: 0.6293386	test: 0.6463849	best: 0.6463849 (8)	total: 179ms	remaining: 22.2s
9:	learn: 0.6230319	test: 0.6418959	best: 0.6418959 (9)	total: 183ms	remaining: 20.4s
10:	learn: 0.6165988	test: 0.6371324	best: 0.6371324 (10)	total: 186ms	remaining: 18.9s
11:	learn: 0.6105294	test: 0.6334278	best: 0.6334278

In [25]:
xgbm_meta_pred: np.ndarray = BASE_MODEL_XGBM.predict_proba(X_ROOT_TEST)
lgbm_meta_pred: np.ndarray = BASE_MODEL_LGBM.predict_proba(X_ROOT_TEST)
cb_meta_pred: np.ndarray   = BASE_CB_MODEL.predict_proba(X_ROOT_TEST)

In [26]:
df_meta: pd.DataFrame = pd.DataFrame({
    'XGBM_PREDICTION' : xgbm_meta_pred[:, 1],
    'LGBM_PREDICTION' : lgbm_meta_pred[:, 1],
    'CB_PREDICTION'   : cb_meta_pred[:, 1],
    'ACTUAL_VALUE'    : Y_ROOT_TEST
})
df_meta

,XGBM_PREDICTION,LGBM_PREDICTION,CB_PREDICTION,ACTUAL_VALUE
127,0.853289,0.947022,0.878905,1
2504,0.013197,0.005113,0.061168,0
634,0.044218,0.036038,0.119815,0
1797,0.760677,0.797002,0.529547,1
2153,0.049030,0.033177,0.176309,0
...,...,...,...,...
880,0.857422,0.854587,0.854766,0
2031,0.002385,0.000824,0.041366,0
1600,0.143663,0.046718,0.195612,0
1581,0.007739,0.006773,0.052485,0


In [27]:
from sklearn.ensemble import GradientBoostingClassifier

META_X = df_meta.drop('ACTUAL_VALUE' , axis = 1).copy()
META_Y = df_meta['ACTUAL_VALUE']

META_X_TRAIN , META_X_TEST, META_Y_TRAIN , META_Y_TEST = train_test_split(
    META_X,
    META_Y,
    train_size = 0.5,
    test_size  = 0.5,
    stratify = META_Y,
    shuffle = True,
    random_state = 13
)
weight_pos: float = (META_Y_TRAIN == 0).sum()/(META_Y_TRAIN == 1).sum()
weight_neg: float = 1.0
weights: np.ndarray = np.where(META_Y_TRAIN == 1, weight_pos , weight_neg)

def obj_for_meta(trial: optuna.Trial) -> float:

    gbm_params: dict = {
        'loss' : 'log_loss',
        'learning_rate' : trial.suggest_float('learning_rate', 0.1 , 0.15),
        'n_estimators'  : trial.suggest_int('n_estimators', 30, 100),
        'subsample'     : trial.suggest_float('subsample', 0.8 , 1),
        'min_samples_split': int(2),
        'min_samples_leaf' : int(1),
        'max_depth'      : trial.suggest_int('max_depth', 3, 8),
        'random_state' : int(13)
    }

    __model: GradientBoostingClassifier = GradientBoostingClassifier(
        **gbm_params
    )
    
    __model.fit(META_X_TRAIN , META_Y_TRAIN, sample_weight = weights)
    validator: StratifiedKFold = StratifiedKFold(
        n_splits= 10 ,
        shuffle= True, 
        random_state= 13
        )
    accuracy: float = cross_val_score(
        __model,
        META_X_TEST,
        META_Y_TEST,
        cv = validator,
        scoring = "accuracy",
        n_jobs = -1,
    ).mean()
    return accuracy

storage_path_gbm: str = "sqlite:///gbm_study.db"
study_gbm = optuna.create_study(
    study_name = "gbm_study_6",
    storage  = storage_path_gbm,
    direction= 'maximize',
    load_if_exists = True
)
study_gbm.optimize(
    func = obj_for_meta,
    n_trials = 50,
    n_jobs = -1,
    show_progress_bar = True 
)

[I 2025-12-25 16:30:37,461] A new study created in RDB with name: gbm_study_6
  0%|          | 0/50 [00:01<?, ?it/s]

[I 2025-12-25 16:30:39,122] Trial 5 finished with value: 0.8054131054131053 and parameters: {'learning_rate': 0.12672104404074147, 'n_estimators': 86, 'subsample': 0.948955737656499, 'max_depth': 6}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-25 16:30:39,220] Trial 2 finished with value: 0.8132478632478632 and parameters: {'learning_rate': 0.12950188114972344, 'n_estimators': 31, 'subsample': 0.893717678431399, 'max_depth': 4}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-25 16:30:39,223] Trial 4 finished with value: 0.7977207977207976 and parameters: {'learning_rate': 0.11066430336549668, 'n_estimators': 41, 'subsample': 0.8935031399806452, 'max_depth': 5}. Best is trial 2 with value: 0.8132478632478632.


Best trial: 2. Best value: 0.813248:   8%|▊         | 4/50 [00:02<00:19,  2.30it/s]

[I 2025-12-25 16:30:39,533] Trial 0 finished with value: 0.8017094017094017 and parameters: {'learning_rate': 0.11784439106160452, 'n_estimators': 77, 'subsample': 0.8387938920076922, 'max_depth': 5}. Best is trial 2 with value: 0.8132478632478632.
[I 2025-12-25 16:30:39,588] Trial 1 finished with value: 0.7864672364672364 and parameters: {'learning_rate': 0.14616165370537598, 'n_estimators': 87, 'subsample': 0.9725549897451385, 'max_depth': 7}. Best is trial 2 with value: 0.8132478632478632.


Best trial: 2. Best value: 0.813248:  10%|█         | 5/50 [00:03<00:19,  2.30it/s]

...

[I 2025-12-25 16:30:47,867] Trial 43 finished with value: 0.7978632478632479 and parameters: {'learning_rate': 0.11463374046299167, 'n_estimators': 93, 'subsample': 0.894340745963955, 'max_depth': 5}. Best is trial 40 with value: 0.8286324786324786.
[I 2025-12-25 16:30:48,001] Trial 46 finished with value: 0.813105413105413 and parameters: {'learning_rate': 0.13790290491794507, 'n_estimators': 79, 'subsample': 0.957637627974519, 'max_depth': 6}. Best is trial 40 with value: 0.8286324786324786.


Best trial: 40. Best value: 0.828632:  98%|█████████▊| 49/50 [00:10<00:00,  6.72it/s]

[I 2025-12-25 16:30:48,322] Trial 49 finished with value: 0.798005698005698 and parameters: {'learning_rate': 0.1378332343803066, 'n_estimators': 94, 'subsample': 0.9389870262033242, 'max_depth': 6}. Best is trial 40 with value: 0.8286324786324786.
[I 2025-12-25 16:30:48,432] Trial 47 finished with value: 0.8055555555555556 and parameters: {'learning_rate': 0.13756728028518345, 'n_estimators': 45, 'subsample': 0.938839233440881, 'max_depth': 6}. Best is trial 40 with value: 0.8286324786324786.


Best trial: 40. Best value: 0.828632: 100%|██████████| 50/50 [00:11<00:00,  4.50it/s]

[I 2025-12-25 16:30:48,575] Trial 48 finished with value: 0.7941595441595442 and parameters: {'learning_rate': 0.13697373042847938, 'n_estimators': 91, 'subsample': 0.9495763018804174, 'max_depth': 6}. Best is trial 40 with value: 0.8286324786324786.


In [28]:
best_params_meta: dict = study_gbm.best_params.copy()
META_MODEL: GradientBoostingClassifier = GradientBoostingClassifier(
    **best_params_meta ,
    min_samples_split = int(2),
    min_samples_leaf = int(1), 
    random_state = int(13))

#
#w1: float = (META_Y == 0).sum()/(META_Y == 1).sum()
#w0: float = 1.0
#weights_2: np.ndarray = np.where(META_Y == 1 , w1 , w0)
#
META_MODEL.fit(
    META_X_TRAIN, 
    META_Y_TRAIN, 
    sample_weight = weights
    )


,loss,'log_loss'
,learning_rate,0.13121904389117106
,n_estimators,36
,subsample,0.8930689961627997
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [29]:
test_df: pd.DataFrame = pd.read_csv('test.csv')
test_df.columns = [
    'date',
    'temp_9am',
    'temp_3pm',
    'min_temp',
    'max_temp',
    'rain_fall',
    'evaporation',
    'sunshine',
    'wind_gust_dir',
    'wind_gust_speed',
    'wind_dir_9am',
    'wind_dir_3pm',
    'wind_speed_9am',
    'wind_speed_3pm',
    'humidity_9am',
    'humidity3pm',
    'pressure_9am',
    'pressure_3pm',
    'cloud_9am',
    'cloud_3pm',
    'id'
]
ids = test_df['id']
test_df = test_df.drop('id' , axis = 1)
test_df['date'] = test_df['date'].map(date_to_days)


In [30]:
cat_columns: list = []
for item in test_df.columns:
    if test_df[item].dtype == object:
        cat_columns.append(item)
print(cat_columns)
for item in cat_columns: print(len(test_df[item].unique()))

['wind_gust_dir', 'wind_dir_9am', 'wind_dir_3pm']
16
16
16


In [31]:
encoder = OneHotEncoder(sparse_output = False , 
                        categories = 'auto', 
                        handle_unknown= 'error',
                        max_categories = 32)

encoder.fit(test_df[cat_columns])
cat_ohed = pd.DataFrame(encoder.transform(test_df[cat_columns]))

In [32]:
test_df = test_df.drop(cat_columns, axis = 1)
test_df = pd.concat([test_df , cat_ohed], axis = 1)
test_df

,date,temp_9am,temp_3pm,min_temp,max_temp,rain_fall,evaporation,sunshine,wind_gust_speed,wind_speed_9am,...,38,39,40,41,42,43,44,45,46,47
0,7396,12.7,17.0,9.0,19.7,0.0,3.2,10.8,31,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7397,11.9,18.6,8.0,18.8,0.0,4.0,10.4,31,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7398,14.4,20.6,11.2,20.8,2.8,5.6,8.0,52,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,7399,13.7,18.3,12.5,19.5,20.4,4.0,5.7,48,19,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7400,15.2,18.7,11.5,21.7,0.0,2.8,9.3,37,17,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,8046,10.5,17.9,8.6,19.6,0.0,2.0,7.8,37,22,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
651,8047,11.0,18.7,9.3,19.2,0.0,2.0,9.2,30,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652,8048,10.2,17.3,9.4,17.7,0.0,2.4,2.7,24,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
653,8049,12.4,19.0,10.1,19.3,0.0,1.4,9.3,43,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
base_xgbm_test_pred: np.ndarray = BASE_MODEL_XGBM.predict_proba(test_df)
base_lgbm_test_pred: np.ndarray = BASE_MODEL_LGBM.predict_proba(test_df)
base_cb_test_pred  : np.ndarray = BASE_CB_MODEL.predict_proba(test_df)

meta_test_df: pd.DataFrame =  pd.DataFrame({
    "XGBM_PREDICTION" : base_xgbm_test_pred[:, 1],
    "LGBM_PREDICTION" : base_lgbm_test_pred[:, 1],
    "CB_PREDICTION"   : base_cb_test_pred[:, 1]
})

In [36]:
final_result: np.ndarray = META_MODEL.predict(meta_test_df)
resultDataFrame: pd.DataFrame = pd.DataFrame({
    'ID' : ids,
    'Predicted' : pd.Series(final_result).map({1 : "yes" , 0 : "no"})
})

In [ ]:
resultDataFrame.to_csv("result.csv" , index = False)